In [1]:
RANDOM_STATE = 1

In [2]:
#!pip install spacy

In [3]:
#!python -m spacy download en

# Imports

In [4]:
import pandas as pd
import time
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth',300)

from nltk.corpus import stopwords
import re
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer

import math

import spacy
import os
#import pyprojroot.here as here

# Import datasets from drive

Note - for the purpose of brevity, data has already been preprocessed and cleaned.

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [66]:
RAW_FOLDER = "data\\raw"

In [35]:
csvs = os.listdir(r"C:\Nga\BT4222\sentiment_analysis_bt4222\data\raw")

In [8]:
csvs

['crowne-plaza.csv',
 'fullerton.csv',
 'grand-mercure-sg-roxy.csv',
 'holiday-inn-express-clarke-quay.csv',
 'hotel-boss.csv',
 'hotel-G.csv',
 'ibis-sg-bencoolen.csv',
 'mbs_total.csv',
 'pan-pacific.csv',
 'paradox-sg-merchant-court.csv',
 'park-regis.csv',
 'parkroyal-collection-marina-bay.csv',
 'swissotel-the-stamford.csv',
 'village-hotel-albert-court-by-far-east-hospitality.csv',
 'village-hotel-changi-by-far-east-hospitality.csv']

In [10]:
#data_path = '/content/drive/MyDrive/BT4222/data/raw/mbs_total.csv'
#df = pd.read_csv(data_path)
#df.head(10)

# Clean Data

## Clean Ratings column

In [84]:
def rating_clean(rating1, rating2):
    try:
        if not math.isnan(rating1):
            return rating1
    except:
        try:
            if not math.isnan(rating2):
                return rating2
        except:
            return None
    return rating2

def valid_rating(rating):
    try:
        if math.isnan(rating):
            return False
        return True
    except:
        return False

def classify_rating(rating):
    if rating>=4:
        return "Positive"
    if rating<=2:
        return "Negative"
    if rating == 3:
        return "Neutral"
    return None

In [12]:
#df["rating"] = df.apply(lambda row: rating_clean(row['rating1'], row['rating2']), axis = 1)
#df['valid_rating'] = df.apply(lambda row: valid_rating(row['rating']), axis = 1)
#df["label"] = df.apply(lambda row: classify_rating(row['rating']), axis = 1)
#df["label"].value_counts()

## Clean review text

In [13]:
# preprocess function
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
spacy_lemmatizer = spacy.load('en_core_web_sm', disable=['parser','ner'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ngalh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
def preprocess_text(title, review):
    text = str(title) + " " + str(review)
    # lower text
    text = text.lower()
    # Remove newline characters
    text = text.replace('\\n',' ').replace('\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('\\', ' ')
    # Remove punctuation and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Remove multiple spaces
    text = re.sub(r'\s+',' ', text)
    # lemmatize
    text = spacy_lemmatizer(text)
    text = [token.lemma_ for token in text]
    # Remove stop words
    text = ' '.join([word for word in text if word not in stop_words])
    # tokenization done below, so no need to do it here.
    return text

In [15]:
#df["cleaned_review"] = df.apply(lambda row: preprocess_text(row['review_title'], row['review_text']), axis = 1)

## Cleaning Raw title and review, and date

In [16]:
# combine raw title and review
def combine(r):
  return str(r['review_title']) + " " + str(r['review_text'])

In [17]:
#df["combined_review"] = df.apply(lambda row: combine(row), axis = 1)
#df['date'] = df['date_of_stay'].str.split(':').str[1]
#df["date"] = pd.to_datetime(df["date"])

## Specify Covid

In [18]:
from datetime import date, timedelta, datetime

In [19]:
covid_start = datetime(2020, 1, 29, 0, 0)
covid_end = datetime(2022, 4, 1, 0, 0, 0)
def get_period(t):
    if pd.isnull(t):
        return None
    if t - covid_start < timedelta(0):
        return "PreCovid"
    elif t-covid_end >= timedelta(0):
        return "PostCovid"
    return "Covid"

In [20]:
#df["covid"] = df.apply(lambda row: get_period(row["date"]), axis = 1)

In [21]:
#df.head(10)

# Export Cleaned Data

In [67]:
SAVE_FOLDER = 'data\\cleaned'

In [68]:
ROOT = r'C:\Nga\BT4222\sentiment_analysis_bt4222'

In [86]:
csvs

['crowne-plaza.csv',
 'fullerton.csv',
 'grand-mercure-sg-roxy.csv',
 'holiday-inn-express-clarke-quay.csv',
 'hotel-boss.csv',
 'hotel-G.csv',
 'ibis-sg-bencoolen.csv',
 'mbs_total.csv',
 'pan-pacific.csv',
 'paradox-sg-merchant-court.csv',
 'park-regis.csv',
 'parkroyal-collection-marina-bay.csv',
 'swissotel-the-stamford.csv',
 'village-hotel-albert-court-by-far-east-hospitality.csv',
 'village-hotel-changi-by-far-east-hospitality.csv']

In [85]:
def clean_data(csv):
    print(csv)
    data_path = ROOT + "\\"+RAW_FOLDER + "\\" + csv
    df = pd.read_csv(data_path)
    df["rating"] = df.apply(lambda row: rating_clean(row['rating1'], row['rating2']), axis = 1)
    df['valid_rating'] = df.apply(lambda row: valid_rating(row['rating']), axis = 1)
    df["label"] = df.apply(lambda row: classify_rating(row['rating']), axis = 1)
    df["cleaned_review"] = df.apply(lambda row: preprocess_text(row['review_title'], row['review_text']), axis = 1)
    df["combined_review"] = df.apply(lambda row: combine(row), axis = 1)
    df['date'] = df['date_of_stay'].str.split(':').str[1]
    df["date"] = pd.to_datetime(df["date"])
    df["covid"] = df.apply(lambda row: get_period(row["date"]), axis = 1)
    df.to_csv(f"{ROOT}\{SAVE_FOLDER}\cleaned_{csv}")

In [88]:
for csv in ['village-hotel-changi-by-far-east-hospitality.csv']:
    clean_data(csv)

village-hotel-changi-by-far-east-hospitality.csv


In [ ]:
#df.to_csv("/content/drive/MyDrive/BT4222/data/cleaned/cleaned_mbs_reviews.csv")